In [19]:
import os
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)


In [20]:
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/pet_adoptions.sqlite"
db = SQLAlchemy(app)


D:\Users\Black\Anaconda3\envs\PythonData2\lib\site-packages\flask_sqlalchemy\__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [21]:
Base = automap_base()


In [22]:
Base.prepare(db.engine, reflect=True)


In [23]:
breed_file= "static/data/breed_labels.csv"
breed_df= pd.read_csv(breed_file)
breed_df.head()

,BreedID,Type,BreedName
0,1,1,Affenpinscher
1,2,1,Afghan Hound
2,3,1,Airedale Terrier
3,4,1,Akbash
4,5,1,Akita


In [24]:

breed_df.describe()


,BreedID,Type
count,307.000000,307.000000
mean,154.000000,1.214984
std,88.767487,0.411482
min,1.000000,1.000000
25%,77.500000,1.000000
50%,154.000000,1.000000
75%,230.500000,1.000000
max,307.000000,2.000000


In [25]:
engine = create_engine("sqlite:///pet_adoptions.sqlite", echo=False)


In [26]:
engine.table_names()

[]

In [27]:
color_file= "static/data/color_labels.csv"
color_df= pd.read_csv(color_file)
color_df.head()

,ColorID,ColorName
0,1,Black
1,2,Brown
2,3,Golden
3,4,Yellow
4,5,Cream


In [28]:
color_df.describe()

,ColorID
count,7.000000
mean,4.000000
std,2.160247
min,1.000000
25%,2.500000
50%,4.000000
75%,5.500000
max,7.000000


In [29]:
breed_df.to_sql(name='breed_labels', con=engine, if_exists='append', index=True)
color_df.to_sql(name='color_labels', con=engine, if_exists='append', index=True)



In [30]:
engine.table_names()

['breed_labels', 'color_labels']

In [31]:
train_file= "static/data/train.csv"
train_df= pd.read_csv(train_file)
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [32]:
train_df.describe()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt,AdoptionSpeed
count,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000
mean,1.457614,10.452078,265.272594,74.009738,1.776162,2.234176,3.222837,1.882012,1.862002,1.467485,1.731208,1.558727,1.914227,1.036617,1.576069,21.259988,41346.028347,0.056760,3.889215,2.516441
std,0.498217,18.155790,60.056818,123.011575,0.681592,1.745225,2.742562,2.984086,0.547959,0.599070,0.667649,0.695817,0.566172,0.199535,1.472477,78.414548,32.444153,0.346185,3.487810,1.177265
min,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,41324.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,265.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,41326.000000,0.000000,2.000000,2.000000
50%,1.000000,3.000000,266.000000,0.000000,2.000000,2.000000,2.000000,0.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,0.000000,41326.000000,0.000000,3.000000,2.000000
75%,2.000000,12.000000,307.000000,179.000000,2.000000,3.000000,6.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,41401.000000,0.000000,5.000000,4.000000
max,2.000000,255.000000,307.000000,307.000000,3.000000,7.000000,7.000000,7.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,20.000000,3000.000000,41415.000000,8.000000,30.000000,4.000000


In [33]:
train_df.to_sql(name='train', con=engine, if_exists='append', index=True)

In [34]:
postcode_file= "static/data/geocoded_state_labels.csv"
postcode_df= pd.read_csv(postcode_file)
postcode_df.head()

,StateID,StateName,LatLong
0,41336,Johor,"1.934400, 103.358727"
1,41325,Kedah,"6.155672, 100.569649"
2,41367,Kelantan,"6.125397, 102.238068"
3,41401,Kuala Lumpur,"3.1412, 101.68653"
4,41415,Labuan,"5.285153, 115.247787"


In [35]:
postcode_df.describe()

,StateID
count,15.000000
mean,41349.733333
std,29.001149
min,41324.000000
25%,41328.500000
50%,41336.000000
75%,41364.000000
max,41415.000000


In [36]:
postcode_df.to_sql(name='state_labels', con=engine, if_exists='append', index=True)

In [37]:
color_id1=engine.execute("SELECT State from train where Color1 = 1")

In [38]:

color_id7=engine.execute("SELECT State from train where Color1 = 7")

In [39]:

color_by_postcode= engine.execute("Select State, Color1, COUNT(COLOR1) from train Group By State, Color1")

In [40]:
print(color_by_postcode)
for value in color_by_postcode:
    print(value)

(41324, 1, 53)
(41324, 2, 45)
(41324, 3, 9)
(41324, 4, 5)
(41324, 5, 8)
(41324, 6, 5)
(41324, 7, 12)
(41325, 1, 63)
(41325, 2, 22)
(41325, 3, 12)
(41325, 4, 1)
(41325, 5, 7)
(41325, 6, 2)
(41325, 7, 3)
(41326, 1, 4351)
(41326, 2, 2241)
(41326, 3, 550)
(41326, 4, 294)
(41326, 5, 525)
(41326, 6, 380)
(41326, 7, 373)
(41327, 1, 415)
(41327, 2, 231)
(41327, 3, 41)
(41327, 4, 32)
(41327, 5, 48)
(41327, 6, 30)
(41327, 7, 46)
(41330, 1, 209)
(41330, 2, 118)
(41330, 3, 17)
(41330, 4, 13)
(41330, 5, 37)
(41330, 6, 15)
(41330, 7, 11)
(41332, 1, 128)
(41332, 2, 63)
(41332, 3, 10)
(41332, 4, 12)
(41332, 5, 19)
(41332, 6, 10)
(41332, 7, 11)
(41335, 1, 47)
(41335, 2, 17)
(41335, 3, 3)
(41335, 4, 3)
(41335, 5, 3)
(41335, 6, 3)
(41335, 7, 9)
(41336, 1, 248)
(41336, 2, 120)
(41336, 3, 27)
(41336, 4, 29)
(41336, 5, 27)
(41336, 6, 26)
(41336, 7, 30)
(41342, 1, 11)
(41342, 2, 1)
(41342, 4, 1)
(41345, 1, 9)
(41345, 2, 6)
(41345, 4, 5)
(41345, 6, 2)
(41361, 1, 7)
(41361, 2, 4)
(41361, 3, 4)
(41361, 4, 5)
(4

In [41]:
breed_by_postcode= engine.execute("Select State, Breed1, COUNT(Breed1) from train GROUP BY State, Breed1 ")


In [42]:
for value in breed_by_postcode:
    print(value)

(41324, 98, 1)
(41324, 109, 2)
(41324, 141, 1)
(41324, 152, 1)
(41324, 179, 5)
(41324, 190, 1)
(41324, 195, 1)
(41324, 205, 2)
(41324, 218, 1)
(41324, 265, 19)
(41324, 266, 21)
(41324, 285, 6)
(41324, 292, 2)
(41324, 296, 1)
(41324, 299, 1)
(41324, 307, 72)
(41325, 39, 1)
(41325, 103, 3)
(41325, 109, 1)
(41325, 141, 13)
(41325, 152, 1)
(41325, 179, 1)
(41325, 195, 1)
(41325, 205, 1)
(41325, 218, 1)
(41325, 251, 1)
(41325, 254, 1)
(41325, 264, 2)
(41325, 265, 10)
(41325, 266, 29)
(41325, 268, 1)
(41325, 285, 5)
(41325, 289, 1)
(41325, 292, 2)
(41325, 297, 1)
(41325, 307, 34)
(41326, 0, 3)
(41326, 1, 1)
(41326, 3, 1)
(41326, 5, 1)
(41326, 10, 1)
(41326, 15, 1)
(41326, 17, 4)
(41326, 18, 6)
(41326, 19, 4)
(41326, 20, 51)
(41326, 24, 1)
(41326, 25, 1)
(41326, 26, 21)
(41326, 31, 8)
(41326, 32, 2)
(41326, 39, 13)
(41326, 42, 3)
(41326, 44, 7)
(41326, 49, 10)
(41326, 50, 4)
(41326, 56, 1)
(41326, 60, 19)
(41326, 61, 1)
(41326, 64, 1)
(41326, 65, 1)
(41326, 69, 24)
(41326, 70, 11)
(41326, 71,

In [43]:
adoptionspeed_by_postcode= engine.execute("Select State, AdoptionSpeed, COUNT(AdoptionSpeed) from train GROUP BY State, AdoptionSpeed ")


In [44]:
for value in adoptionspeed_by_postcode:
    print(value)

(41324, 0, 4)
(41324, 1, 18)
(41324, 2, 23)
(41324, 3, 12)
(41324, 4, 80)
(41325, 0, 3)
(41325, 1, 14)
(41325, 2, 34)
(41325, 3, 23)
(41325, 4, 36)
(41326, 0, 246)
(41326, 1, 1877)
(41326, 2, 2435)
(41326, 3, 2004)
(41326, 4, 2152)
(41327, 0, 8)
(41327, 1, 122)
(41327, 2, 216)
(41327, 3, 197)
(41327, 4, 300)
(41330, 0, 3)
(41330, 1, 48)
(41330, 2, 111)
(41330, 3, 117)
(41330, 4, 141)
(41332, 0, 4)
(41332, 1, 36)
(41332, 2, 63)
(41332, 3, 42)
(41332, 4, 108)
(41335, 0, 3)
(41335, 1, 29)
(41335, 2, 14)
(41335, 3, 16)
(41335, 4, 23)
(41336, 0, 23)
(41336, 1, 113)
(41336, 2, 136)
(41336, 3, 103)
(41336, 4, 132)
(41342, 0, 1)
(41342, 1, 1)
(41342, 3, 2)
(41342, 4, 9)
(41345, 0, 1)
(41345, 1, 6)
(41345, 2, 3)
(41345, 3, 4)
(41345, 4, 8)
(41361, 1, 9)
(41361, 2, 2)
(41361, 3, 6)
(41361, 4, 9)
(41367, 0, 2)
(41367, 1, 3)
(41367, 2, 3)
(41367, 3, 1)
(41367, 4, 6)
(41401, 0, 112)
(41401, 1, 814)
(41401, 2, 996)
(41401, 3, 731)
(41401, 4, 1192)
(41415, 2, 1)
(41415, 3, 1)
(41415, 4, 1)


In [45]:
engine.table_names()

['breed_labels', 'color_labels', 'state_labels', 'train']